<a href="https://colab.research.google.com/github/vineet-codes/defi-exploration/blob/main/simulation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import matplotlib.pyplot as plt
import pandas as pd
import itertools

In [2]:
def get_usernames():
  inputstring = 'ABCDEFGHIJKLMNOPQRSTUVWXYZabscdefghijklmnopqrstuvwxyz0123456789'
  # print(len(inputstring))
  username_list =list(set([inputstring[x:y] for x, y in itertools.combinations(range(len(inputstring) + 1), r = 2)]))
  return username_list

len(get_usernames())

2015

In [59]:
class Vault:

  def __init__(self):
    # pool is of the form {user: [(amount_1, tick_1), (amount_2, tick_2)]}
    self.pool = {}
    # self.rewards_pool = rewards_pool
    # tick is used for having a contcept of time in our analysis
    self.tick = 0

  def update_tick(self, step):
    self.tick += step

  def balanceOf(self, user):
    user_txns = self.pool.get(user,[])
    if(len(user_txns) == 0):
      return 0 #f'It seems {user} has made no txns in to the Vault'
    return sum([amount for (amount, _) in user_txns])
    

  def shareOfUser(self, user):
    """
    return: users shares of the total pool
    """
    return self.balanceOf(user) / self.tvl()

  def deposit(self, amount, user):
    """
    return: tuple (user, updated balance)
    """
    if (user in self.pool.keys()):
      user_txns = self.pool.get(user, []) 
      user_txns.append((amount, self.tick))
      self.pool[user] = user_txns
    else:
      self.pool[user] = [(amount, self.tick)]
    return user, self.pool[user]

  def withdraw(self, withdraw_amount, user):
    """
    return: tuple (user, updated balance)
    """
    if(user not in self.pool.keys()):
      return f'{user} has not deposited in the vault'
    
    if(self.balanceOf(user) < withdraw_amount):
      return f'Not enough balance, {user} can one withdraw upto: {self.balanceOf(user)}'
    
    # db update
    # self.pool[user] = self.pool[user] - withdraw_amount
    self.pool[user].append((-1* withdraw_amount, self.tick))

    return user,self.balanceOf(user)

  def tvl(self):
    result = 0
    for user, txns in self.pool.items():
      user_balance = self.balanceOf(user)
      result += user_balance
    return result
      
    return sum([v for _,v in self.pool.items()])
  
  def get_pool(self):
    return self.pool



class Stakes:

  def __init__(self):
    self.pool = {}
    # self.rewards_pool = rewards_pool
    self.tick = 0

  def update_tick(self, step):
    self.tick += step

  def stake(self, user, amount, duration):
    """
    duratrion: in months
    """
    # assert self.tick == tick

    user_stakes = self.pool.get(user, [])
    # print(user_stakes)
    user_stakes.append((amount, duration, self.tick))
    self.pool[user] = user_stakes
    # self.rewards_pool.tick()
    return user, self.pool.get(user, [])

  def unstake(self):
    pass


  def get_pool(self):
    return self.pool

  def pool_size(self):
    return sum([amount*duration for user in self.pool.keys() for amount, duration, _ in self.pool[user]])

  def shareOfUser(self, user):
    totalPoolSize = self.pool_size()
    userPoolSize = sum([amount*duration for amount, duration, _ in self.pool.get(user, [(0,0,0)])])
    return userPoolSize / totalPoolSize



class Rewards:

  def __init__(self, vault, stakes, 
               weights=[0.2,0.8], 
               vault_stake_reward_split = [0.75, 0.25], 
               yearly_emissions = 3891931,
               yop_token_price = 1):
    
    self.tick = 0
    
    self.yearly_emissions = yearly_emissions

    self.vault = vault
    self.stakes = stakes

    self.weights = weights
    self.vault_stake_reward_split = vault_stake_reward_split
    self.yop_token_price = yop_token_price

  
  def update_tick(self, step):
    self.tick += step

  def calculate_apys(self):
    all_vault_users = self.vault.get_pool().keys() 
    all_stakes_users = self.stakes.get_pool().keys() 

  def calculate_rewards(self):

    result = {}
    
    total_power = 0
    for user in list(set(self.vault.pool.keys())):
      vault_boost_multiplier = self.weights[0]*self.vault.shareOfUser(user) + self.weights[1]*self.stakes.shareOfUser(user)
      total_power += vault_boost_multiplier * self.vault.balanceOf(user)

    print(total_power)

    for user in list(set(self.vault.pool.keys())):
      # print("=========")
      vault_boost_multiplier = self.weights[0]*self.vault.shareOfUser(user) + self.weights[1]*self.stakes.shareOfUser(user)
      # print(f'{user} vault boost multiplier is: {vault_boost_multiplier}')
      # x = (vault_boost_multiplier * self.vault.balanceOf(user) / total_power) * ((self.vault_stake_reward_split[0]/4)*self.yearly_emissions) * (self.yop_token_price)
      x = ((vault_boost_multiplier * self.vault.balanceOf(user)) / total_power) * ((0.2)*self.yearly_emissions) * (self.yop_token_price)
      # print(f"{user} is getting {(x / self.vault.balanceOf(user))*100}% APY in the vault")
      result[user] = (x / self.vault.balanceOf(user))*100
    return result
 
    
  
  def tick_fn(self, step):

    print(f"===Tick: {self.tick}====")
    print(self.calculate_rewards())
    print("Vault pool",self.vault.pool)
    print("Stake Pool", self.stakes.pool)
    # print("========")

    self.update_tick(step)
    self.vault.update_tick(step)
    self.stakes.update_tick(step)
    
      

  


# v = Vault()
# print(v.tvl())
# print(v.balanceOf('dfg'))
# v.deposit(5000, 'e')
# print(v.tvl())

class User:

  def __init__(self, name):
    self.name = name

  def __repr__(self):
    return f'{self.name}'

  def vault_rewards(self):
    pass

  def stake_rewards(self):
    pass

In [60]:
usdc_vault = Vault()
yop_stakes = Stakes()
vineet = User("Vineet")
john = User("John")
anar = User("Anar")

reward_calculator = Rewards(usdc_vault, yop_stakes)

print(reward_calculator.tick_fn(1))

# users make deposit into vault
usdc_vault.deposit(50_000, vineet)
yop_stakes.stake(john, 5000, 12)


reward_calculator.tick_fn(1)

usdc_vault.deposit(50_000, vineet)

reward_calculator.tick_fn(1)

usdc_vault.deposit(60_000, john)

# users stake some yop
yop_stakes.stake(vineet, 5000, 12)
reward_calculator.tick_fn(1)

yop_stakes.stake(vineet, 5000, 12)
reward_calculator.tick_fn(1)


reward_calculator.tick_fn(1)

===Tick: 0====
0
{}
Vault pool {}
Stake Pool {}
None
===Tick: 1====
10000.0
{Vineet: 1556.7724000000003}
Vault pool {Vineet: [(50000, 1)]}
Stake Pool {John: [(5000, 12, 1)]}
===Tick: 2====
20000.0
{Vineet: 778.3862000000001}
Vault pool {Vineet: [(50000, 1), (50000, 2)]}
Stake Pool {John: [(5000, 12, 1)]}
===Tick: 3====
81000.0
{Vineet: 504.50957407407407, John: 456.4610432098767}
Vault pool {Vineet: [(50000, 1), (50000, 2)], John: [(60000, 3)]}
Stake Pool {John: [(5000, 12, 1)], Vineet: [(5000, 12, 3)]}
===Tick: 4====
86333.33333333333
{Vineet: 593.5570444015445, John: 308.0485926640927}
Vault pool {Vineet: [(50000, 1), (50000, 2)], John: [(60000, 3)]}
Stake Pool {John: [(5000, 12, 1)], Vineet: [(5000, 12, 3), (5000, 12, 4)]}
===Tick: 5====
86333.33333333333
{Vineet: 593.5570444015445, John: 308.0485926640927}
Vault pool {Vineet: [(50000, 1), (50000, 2)], John: [(60000, 3)]}
Stake Pool {John: [(5000, 12, 1)], Vineet: [(5000, 12, 3), (5000, 12, 4)]}


In [61]:
usdc_vault.pool, usdc_vault.balanceOf(anar)

({John: [(60000, 3)], Vineet: [(50000, 1), (50000, 2)]}, 0)

In [62]:
usdc_vault.shareOfUser(vineet) + usdc_vault.shareOfUser(john) == 1

True

In [63]:
usdc_vault.withdraw(100000, vineet), usdc_vault.balanceOf(vineet), usdc_vault.tvl(), yop_stakes.get_pool()

((Vineet, 0),
 0,
 60000,
 {John: [(5000, 12, 1)], Vineet: [(5000, 12, 3), (5000, 12, 4)]})

In [64]:
def simultaion_setup(steps = 100):

  # create vault and stake pool
  usdc_vault = Vault()
  yop_stakes = Stakes()
  rewards = Rewards(usdc_vault, yop_stakes)

  # create users
  vineet, john, anar = User("Vineet"), User("John"), User("Anar")

  # make some txns in vaults and stakes in some ticks
  # tick
  rewards.tick_fn(1)

  # users make deposit into vault
  usdc_vault.deposit(670_000, vineet)
  yop_stakes.stake(john, 100_000, 5)

  #tick 
  rewards.tick_fn(1)
  
  usdc_vault.deposit(2_000_000, john)

  # users stake some yop
  # yop_stakes.stake(vineet, 5000, 12)
  rewards.tick_fn(1)

  usdc_vault.deposit(5000, anar)

  yop_stakes.stake(anar, 50000, 5)
  rewards.tick_fn(1)

  # yop_stakes.stake(anar, 5000, 12)
  rewards.tick_fn(1)



  return rewards, usdc_vault, yop_stakes, [vineet, john, anar]

In [65]:
rewards, usdc_vault, yop_stakes, [vineet, john, anar] = simultaion_setup()

===Tick: 0====
0
{}
Vault pool {}
Stake Pool {}
===Tick: 1====
134000.0
{Vineet: 116.1770447761194}
Vault pool {Vineet: [(670000, 1)]}
Stake Pool {John: [(100000, 5, 1)]}
===Tick: 2====
1933250.936329588
{Vineet: 2.0206934584581293, John: 38.24237769141653}
Vault pool {Vineet: [(670000, 1)], John: [(2000000, 2)]}
Stake Pool {John: [(100000, 5, 1)]}
===Tick: 3====
1400629.9065420562
{Vineet: 2.7838943172431097, John: 37.94960412560259, Anar: 14.840511248388019}
Vault pool {Vineet: [(670000, 1)], John: [(2000000, 2)], Anar: [(5000, 3)]}
Stake Pool {John: [(100000, 5, 1)], Anar: [(50000, 5, 3)]}
===Tick: 4====
1400629.9065420562
{Vineet: 2.7838943172431097, John: 37.94960412560259, Anar: 14.840511248388019}
Vault pool {Vineet: [(670000, 1)], John: [(2000000, 2)], Anar: [(5000, 3)]}
Stake Pool {John: [(100000, 5, 1)], Anar: [(50000, 5, 3)]}


In [66]:
usdc_vault.pool

{Anar: [(5000, 3)], John: [(2000000, 2)], Vineet: [(670000, 1)]}

In [57]:
yop_stakes.pool

{Anar: [(50000, 5, 3)], John: [(100000, 5, 1)]}

In [58]:
apys = rewards.calculate_rewards()
apys

1400629.906542056


{Anar: 14.840511248388022, John: 37.9496041256026, Vineet: 2.78389431724311}

In [116]:
common_users = list(set(rewards.vault.pool.keys()).intersection(set(rewards.stakes.pool.keys())))

In [117]:
common_users

[Anar, John]

In [118]:
yop_token_price = 1

In [119]:
total_power = 0
for user in list(set(rewards.stakes.pool.keys())):
  vault_boost_multiplier = rewards.weights[0]*rewards.vault.shareOfUser(user) + rewards.weights[1]*rewards.stakes.shareOfUser(user)
  total_power += vault_boost_multiplier * rewards.vault.balanceOf(user)

In [126]:
for user in list(set(rewards.vault.pool.keys())):
  print("=========")
  vault_boost_multiplier = rewards.weights[0]*rewards.vault.shareOfUser(user) + rewards.weights[1]*rewards.stakes.shareOfUser(user)
  # print(f'{user} vault boost multiplier is: {vault_boost_multiplier}')
  # x = (vault_boost_multiplier * rewards.vault.balanceOf(user) / total_power) * ((rewards.vault_stake_reward_split[0]/4)*rewards.yearly_emissions) * (yop_token_price)
  x = ((vault_boost_multiplier * rewards.vault.balanceOf(user)) / total_power) * ((0.2)*rewards.yearly_emissions) * (yop_token_price)
  print(f"{user} is getting {(x / rewards.vault.balanceOf(user))*100}% APY in the vault")
 


Vineet is getting 2.8522411930307188% APY in the vault
Anar is getting 15.204857902151318% APY in the vault
John is getting 38.881297855244625% APY in the vault


In [99]:
0.2181818181818182 + 0.5818181818181818 + 0.2

1.0

In [102]:
total_power

404436.36363636365